In [1]:
import requests
import pandas as pd
import io
import yfinance as yf
import altair as alt
import time

In [2]:
def crawl_price(i):
    stock_name = yf.Ticker(i)
    hist_tempt = pd.DataFrame(stock_name.history(start = "2020-01-01"))
    hist_tempt.reset_index(level=0, inplace=True)
    price = hist_tempt['Close']
    date = hist_tempt['Date']

    price_hist = {'Date': date, '%s' % i: price}
    price_hist_df = pd.DataFrame(price_hist).set_index('Date')
    time.sleep(1)
    return price_hist_df

def crawl_company_info(list):
    sector_list = []
    country_list = []
    fullTimeEmployees_list = []
    enterpriseValue_list = []
    earningsQuarterlyGrowth_list = []
    profitMargins_list = []
    longName_list = []
    for i in list:
        stock_name = yf.Ticker('%s' % i)
        info = stock_name.info

        if 'fullTimeEmployees' in info:   # some companies don't have this data
            fullTimeEmployees = info[ 'fullTimeEmployees']
        else:
            fullTimeEmployees = None

        if 'sector' in info:   # some companies don't have this data
            sector = info['sector']
        else:
            sector = None
        
        if 'enterpriseValue' in info:   # some companies don't have this data
            enterpriseValue = info['enterpriseValue']
        else:
            enterpriseValue = None
            
        longName = info['longName']

        if 'country' in info:   # some companies don't have this data
            country = info['country']
        else:
            country = None
            
        if 'earningsQuarterlyGrowth' in info:   # some companies don't have this data
            earningsQuarterlyGrowth = info['earningsQuarterlyGrowth']
        else:
            earningsQuarterlyGrowth = None
            
        if 'profitMargins' in info:   # some companies don't have this data
            profitMargins = info['profitMargins']
        else:
            profitMargins = None
            
        country_list.append(country)
        fullTimeEmployees_list.append(fullTimeEmployees)
        longName_list.append(longName)
        profitMargins_list.append(profitMargins)
        earningsQuarterlyGrowth_list.append(earningsQuarterlyGrowth)
        sector_list.append(sector)
        enterpriseValue_list.append(enterpriseValue)
        
    company_info_df = pd.DataFrame({
        'Name': list,
        'LongName': longName_list,
        'Country': country_list,
        'Sector': sector_list,
        'FullTimeEmployees': fullTimeEmployees_list,
        'EnterpriseValue': enterpriseValue_list,
        'EarningsQuarterlyGrowth': earningsQuarterlyGrowth_list,
        'ProfitMargins': profitMargins_list
        })
    time.sleep(0.1)
    return company_info_df

def crawl_index_info(list):
    country_list = []
    for i in list:
        stock_name = yf.Ticker('%s' % i)
        country = stock_name.info['market'][:2]
        country_list.append(country)
    index_info_df = pd.DataFrame({
        'Name': list,
        'Country': country_list
        })
    time.sleep(1)
    return index_info_df

In [45]:
def crawl_price_vertical(i):
    stock_name = yf.Ticker(i)
    hist_tempt = pd.DataFrame(stock_name.history(start = "2020-01-01"))
    hist_tempt.reset_index(level=0, inplace=True)
    price = hist_tempt['Close']
    date = hist_tempt['Date']

    price_hist = {'Date': date, 'Price': price, 'Name': i}
    price_hist_df = pd.DataFrame(price_hist)
    time.sleep(1)
    return price_hist_df

In [ ]:
# price history 纵向merge

world_index_hist_vertical = crawl_price_vertical('399001.SZ')

for i in world_index_stock_name_list:
    price_temp = crawl_price_vertical(i)
    world_index_hist_vertical = pd.concat([world_index_hist_vertical, price_temp])
world_index_hist_vertical
world_index_hist_vertical.to_csv('World Index History Vertical.csv')

# <center>Worldwide Stock Index<center>

In [3]:
url_indices = "https://finance.yahoo.com/world-indices/"
response_indices = requests.get(url_indices)

import io
f_indices = io.StringIO(response_indices.text)
dfs_indices = pd.read_html(f_indices)
world_index = dfs_indices[0]
world_index_stock_name_list= list(world_index['Symbol'])

world_index_stock_name_list = world_index_stock_name_list + ['^OMX', '^IBEX', 'FTSEMIB.MI', '^AEX',
                                   'XU100.IS', '^ATX', '^OMXC25', '^OMXH25',
                                   '^ISEQ', 'PSI20.LS', 'FPXAA.PR', 'WIHUN.FGI',
                                   'CRE.VI', 'WICAN.FGI', 'WIITA.FGI',
                                   'WIESP.FGI', 'WIBRA.FGI', 'WICZH.FGI', 
                                  'WIGRC.FGI', 'WIMAL.FGI', '^KLSE', '^MXX',
                                  'WITHA.FGI', 'WITUR.FGI', 'WICHL.FGI',
                                  'WICOL.FGI', '^DJEG20E', 'WIIDN.FGI',
                                  '^DJIMKW', 'WIPAK.FGI', 'WIPHL.FGI',
                                  'DUAE.FGI', 'FTCUSAUN.FGI', 'WIQAT.FGI',
                                  '^OMXI10', '^NQMAEUR', 'FVTT.FGI', 'WIPER.FGI']    # the index of Sweden is not included

In [99]:
# information

world_index_info = crawl_index_info(world_index_stock_name_list)
world_index_info.to_csv('World Index Info.csv')

# country abbrev. will be converted into a more readable form.

In [5]:
# price history

world_index_hist = crawl_price('399001.SZ')

for i in world_index_stock_name_list:
    price_temp = crawl_price(i)
    world_index_hist = pd.concat([world_index_hist, price_temp], axis = 1)
# world_index_hist.to_csv('World Index History.csv')

- ^IPSA: No data found for this date range, symbol may be delisted


History of some indices were added mannually --> see Data Mammual

### calculate change rate

In [6]:
price_indix_1902 = world_index_hist[35:36]     # price on 19 Feb. 
price_indix_1210 = world_index_hist[203:204]    # price on 12 Oct. 
price_indix_1902

,399001.SZ,^GSPC,^DJI,^IXIC,^NYA,^XAX,^BUK100P,^RUT,^VIX,^FTSE,...,^DJIMKW,WIPAK.FGI,WIPHL.FGI,DUAE.FGI,FTCUSAUN.FGI,WIQAT.FGI,^OMXI10,^NQMAEUR,FVTT.FGI,WIPER.FGI
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-19,11235.599609,3386.149902,29348.029297,9817.179688,14087.129883,2425.699951,12627.759766,1692.569946,14.38,7457.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
price_indix_price_1210 = pd.concat([price_indix_1902, price_indix_1210]).dropna(axis = 1)
price_indix_price_1210.reset_index(drop=True, inplace=True)

In [8]:
change_rate_indices_1210 = price_indix_price_1210.pct_change()
change_rate_indices_1210 = change_rate_indices_1210.loc[1].to_frame().reset_index().rename(columns = {'index': 'Name', 1 : 'change_rate'})
indices_df_1210 = change_rate_indices_1210.(world_index_info, on='Name')
indices_df_1210

,Name,change_rate,Country
0,399001.SZ,0.220057,cn
1,399001.SZ,0.220057,cn
2,^GSPC,0.043728,us
3,^DJI,-0.017395,us
4,^IXIC,0.209743,us
5,^NYA,-0.054110,us
6,^XAX,-0.172610,us
7,^BUK100P,-0.952728,dr
8,^RUT,-0.025712,us
9,^VIX,0.743394,us


As we can see there are more than one indices describing the US stock market. We will use the average result.


In [9]:
change_rate_indices_by_country_1210 = indices_df_1210.groupby('Country').first().reset_index()
change_rate_indices_by_country_1210['Country'] = change_rate_indices_by_country_1210['Country'].str.upper()
change_rate_indices_by_country_1210

,Country,Name,change_rate
0,AT,^ATX,-0.306241
1,AU,^AXJO,-0.141729
2,BE,^BFX,-0.182134
3,CH,^STOXX50E,-0.146710
4,CN,399001.SZ,0.220057
5,DE,^GDAXI,-0.047182
6,DR,^BUK100P,-0.952728
7,ES,^IBEX,-0.310663
8,FR,^FCHI,-0.185224
9,GB,^FTSE,-0.195199


Then we will replace one country's abbrevation with its full name.

In [20]:
code_lookup = pd.read_csv("country_lookup.csv")

change_rate_indices_by_Country_1210 = change_rate_indices_by_country_1210.merge(code_lookup, left_on='Country', right_on='Alpha-2 code')
# some countries/districts are not interested, so we droped them: Korea, Taiwan
# change_rate_indices_by_Country_1210 = change_rate_indices_by_Country_1210.drop([12, 17, 21, 22])

### Visualization

In [21]:
alt.Chart(change_rate_indices_by_Country_1210).mark_bar().encode(
    alt.X('change_rate:Q', 
          title='Change Rate (19 Feb.- 12 Oct.)'),
    alt.Y('Country_y:O',
          sort = alt.SortField(field = 'change_rate', order ='descending'),
          title='Country'),
    color = 'change_rate',
    tooltip = ['Country_y', 'change_rate']
).properties(
    title = 'World Indices Change Rate VS Country',
    width = 350,
    height = 350
)

alt.Chart(...)

# <center>S&P 500<center>

In [ ]:
SP500_company = pd.read_excel("S&P 500 company list.xlsx")
SP500_company_list = list(SP500_company['Symbol'])
SP500_company_sector = list(SP500_company['Sector'])

In [ ]:
# price history

company_hist = crawl_price('FB')

for i in SP500_company_list:
    price_temp = crawl_price(i)
    company_hist = pd.concat([company_hist, price_temp], axis = 1)
    
company_hist = company_hist.loc[:,~company_hist.columns.duplicated()]    # drop duplicated (FB)
company_hist.to_csv('S&P500 Company History.csv')

In [ ]:
# information

company_info = crawl_company_info(SP500_company_list)
company_info.to_csv('S&P500 Company Info.csv')

# <center>DAX 30<center>

In [ ]:
DAX30_company = pd.read_excel("DAX30 Name List.xlsx")
DAX30_company_list = list(DAX30_company['stock_name.de'])

In [ ]:
# price history

DAX30_company_hist = crawl_price('SAP.DE')

for i in DAX30_company_list:
    price_temp = crawl_price(i)
    DAX30_company_hist = pd.concat([DAX30_company_hist, price_temp], axis = 1)
DAX30_company_hist = DAX30_company_hist.loc[:,~DAX30_company_hist.columns.duplicated()]    # drop duplicated (SAP.DE)
DAX30_company_hist.to_csv('DAX30 Company History.csv')

In [ ]:
# info

DAX30_company_info = crawl_company_info(DAX30_company_list)

we noticed the information for MTX.DE are missing, so we'll add it manually.

In [ ]:
# source: https://www.marketwatch.com/investing/stock/mtx/company-profile?countrycode=de&iso=xfra&mod=mw_quote_tab

DAX30_company_info.loc[30, ['Sector','FullTimeEmployees', 'ProfitMargins']] = ['Industrials', 10660, 0.1033]
DAX30_company_info.to_csv('DAX30 Company Info.csv')

# <center>CSI 300<center>

In [ ]:
CSI300_company = pd.read_excel("CSI 300 China List.xls")
CSI300_company_list = list(CSI300_company['stock_name'])

In [ ]:
# price history

CSI300_company_hist = crawl_price('000001.SZ')

for i in CSI300_company_list:
    price_temp = crawl_price(i)
    CSI300_company_hist = pd.concat([CSI300_company_hist, price_temp], axis = 1)
CSI300_company_hist = CSI300_company_hist.loc[:,~CSI300_company_hist.columns.duplicated()]    # drop duplicated
CSI300_company_hist.to_csv('CSI300 Company History.csv')

In [ ]:
# info

CSI300_company_info = crawl_company_info(CSI300_company_list)
CSI300_company_info.to_csv('CSI300 Company Info.csv')